In [1]:
import sqlite3
import pandas as pd

# 连接到数据库
conn = sqlite3.connect('data/blog.db')

# 查看数据库中所有的表
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()

print("数据库中的表:")
for table in tables:
    print(f"- {table[0]}")

print("\n" + "="*50 + "\n")

# 查看每个表的结构和内容
for table in tables:
    table_name = table[0]
    print(f"表名: {table_name}")
    
    # 查看表结构
    cursor.execute(f"PRAGMA table_info({table_name});")
    columns = cursor.fetchall()
    print("表结构:")
    for col in columns:
        print(f"  {col[1]} ({col[2]}) - {'NOT NULL' if col[3] else 'NULL'} - {'PK' if col[5] else ''}")
    
    # 查看表内容
    print(f"\n表内容 (前10行):")
    df = pd.read_sql_query(f"SELECT * FROM {table_name} LIMIT 10", conn)
    print(df.to_string())
    
    # 显示总行数
    cursor.execute(f"SELECT COUNT(*) FROM {table_name}")
    count = cursor.fetchone()[0]
    print(f"\n总行数: {count}")
    
    print("\n" + "="*50 + "\n")

# 关闭连接
conn.close()


数据库中的表:
- users
- sqlite_sequence
- comments


表名: users
表结构:
  id (INTEGER) - NULL - PK
  email (TEXT) - NOT NULL - 
  password_hash (TEXT) - NOT NULL - 
  username (TEXT) - NULL - 
  created_at (DATETIME) - NULL - 
  updated_at (DATETIME) - NULL - 

表内容 (前10行):
   id  email                                                 password_hash username           created_at           updated_at
0   1  tom@q  $2b$10$B0RCeyiu8jN3pSoQr.LSnu1vZcU.h.YQoICsnyDVwdkPpmdF3NQM2      Tom  2025-06-11 03:45:03  2025-06-11 03:45:03

总行数: 1


表名: sqlite_sequence
表结构:
  name () - NULL - 
  seq () - NULL - 

表内容 (前10行):
       name  seq
0     users    1
1  comments    2

总行数: 2


表名: comments
表结构:
  id (INTEGER) - NULL - PK
  post_id (TEXT) - NOT NULL - 
  author (TEXT) - NOT NULL - 
  content (TEXT) - NOT NULL - 
  date (DATETIME) - NULL - 
  likes (INTEGER) - NULL - 

表内容 (前10行):
   id                      post_id author    content                 date  likes
0   1  tech/web-development-trends    Tom  Hello 

In [2]:
# 更详细的数据探索
import sqlite3
import pandas as pd

conn = sqlite3.connect('data/comments.db')

# 如果您想查看特定表的详细信息，可以运行以下代码
# 例如，假设有一个名为 'comments' 的表

try:
    # 查看所有评论数据
    print("所有评论数据:")
    df_comments = pd.read_sql_query("SELECT * FROM comments ORDER BY id DESC", conn)
    print(df_comments.to_string())
    
    print(f"\n评论总数: {len(df_comments)}")
    
    # 按状态统计
    if 'status' in df_comments.columns:
        print("\n按状态统计:")
        status_counts = df_comments['status'].value_counts()
        print(status_counts)
    
    # 最近的评论
    if 'created_at' in df_comments.columns:
        print("\n最近5条评论:")
        recent = df_comments.head(5)[['id', 'name', 'email', 'content', 'created_at']]
        print(recent.to_string())
        
except Exception as e:
    print(f"查询出错: {e}")
    print("让我们先看看实际的表结构...")
    
    cursor = conn.cursor()
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables = cursor.fetchall()
    
    for table in tables:
        table_name = table[0]
        print(f"\n表名: {table_name}")
        cursor.execute(f"SELECT * FROM {table_name} LIMIT 5")
        rows = cursor.fetchall()
        
        # 获取列名
        cursor.execute(f"PRAGMA table_info({table_name})")
        columns = [col[1] for col in cursor.fetchall()]
        
        print(f"列名: {columns}")
        print("示例数据:")
        for row in rows:
            print(row)

conn.close()


所有评论数据:
    id                      post_id     author            content                 date  likes
0   15  tech/web-development-trends       momo  Love this world\n  2025-06-10 17:41:07      1
1   14  tech/web-development-trends       momo            1234125  2025-06-10 17:41:01      0
2   13  tech/web-development-trends       momo                  9  2025-06-10 17:40:58      0
3   12  tech/web-development-trends       momo                  8  2025-06-10 17:40:58      0
4   11  tech/web-development-trends       momo                  7  2025-06-10 17:40:57      0
5   10  tech/web-development-trends       momo                  6  2025-06-10 17:40:56      0
6    9  tech/web-development-trends       momo                  5  2025-06-10 17:40:55      0
7    8  tech/web-development-trends       momo                  4  2025-06-10 17:40:54      0
8    7  tech/web-development-trends       momo                  3  2025-06-10 17:40:53      0
9    6  tech/web-development-trends       momo      